In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import os

# Define paths
img_dir = "/content/drive/MyDrive/GDG_project/Multimodal_dataset_assignment3/images"  # Path to your images folder

# Check if the directory exists
if not os.path.exists(img_dir):
    raise FileNotFoundError(f"The directory {img_dir} does not exist. Please check the path.")

print("Images are located in:", img_dir)

Images are located in: /content/drive/MyDrive/GDG_project/Multimodal_dataset_assignment3/images


In [3]:
!pip install transformers torchvision torch pandas pillow



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from transformers import BertModel, BertTokenizer
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from PIL import Image
import os

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")


In [ ]:
from PIL import Image, UnidentifiedImageError, ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES = True


class HumorMemeDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform

        # Ensure column names match exactly in the CSV
        self.data.columns = self.data.columns.str.lower()  # Convert column names to lowercase

        # Label Mapping (Binary Classification)
        self.humour_mapping = {  # Assign to self.humour_mapping
            "not_humorous": 0,
            "humorous": 1,
            "funny": 1,
            "hilarious": 1
        }

        self.sarcasm_mapping = { # Assign to self.sarcasm_mapping
            "not_sarcastic": 0,
            "sarcastic": 1,
            "twisted_meaning": 1,
            "very_twisted": 1
        }

        self.offensive_mapping = { # Assign to self.offensive_mapping
            "not_offensive": 0,
            "slight": 1,
            "very_offensive": 1,
            "hateful_offensive": 1
        }

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data.iloc[idx]["text_corrected"]
        if not isinstance(text, str):
            text = str(text)
        img_path = os.path.join(self.img_dir, self.data.iloc[idx]["image_name"])

        # Convert label column names to lowercase if needed
        humour_label = self.humour_mapping.get(str(self.data.iloc[idx]["humour"]).lower(), 0)
        sarcasm_label = self.sarcasm_mapping.get(str(self.data.iloc[idx]["sarcasm"]).lower(), 0)
        offensive_label = self.offensive_mapping.get(str(self.data.iloc[idx]["offensive"]).lower(), 0)

        # Load image
        try:
            image = Image.open(img_path).convert("RGB")
        except (UnidentifiedImageError, OSError) as e:
            print(f"Error loading image {img_path}: {e}")
            # Handle the error: skip the image, replace with a placeholder, etc.
            # For example, to skip the image:
            return None

        if self.transform:
            image = self.transform(image)

        # Tokenize text
        encoded_text = tokenizer(text, padding="max_length", truncation=True, max_length=128, return_tensors="pt")

        return {
            "input_ids": encoded_text["input_ids"].squeeze(0),
            "attention_mask": encoded_text["attention_mask"].squeeze(0),
            "image": image,
            "labels": torch.tensor([humour_label, sarcasm_label, offensive_label], dtype=torch.float)
        }

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


In [ ]:
csv_path = "/content/drive/MyDrive/GDG_project/Multimodal_dataset_assignment3/labels.csv"
img_dir = "/content/drive/MyDrive/GDG_project/Multimodal_dataset_assignment3/images"

dataset = HumorMemeDataset(csv_file=csv_path, img_dir=img_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)


In [ ]:
class MultimodalHumorModel(nn.Module):
    def __init__(self):
        super(MultimodalHumorModel, self).__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.resnet = models.resnet50(pretrained=True)

        # Remove the last classification layer of ResNet
        self.resnet = nn.Sequential(*list(self.resnet.children())[:-1])

        # Fully connected layers for fusion
        self.fc = nn.Linear(768 + 2048, 3)  # 768 (BERT) + 2048 (ResNet) -> 3 binary output labels

    def forward(self, input_ids, attention_mask, image):
        # Text features from BERT
        text_features = self.bert(input_ids, attention_mask=attention_mask).pooler_output  # (batch_size, 768)

        # Image features from ResNet
        image_features = self.resnet(image)  # (batch_size, 2048, 1, 1)
        image_features = image_features.view(image_features.size(0), -1)  # Flatten -> (batch_size, 2048)

        # Concatenate text and image features
        combined_features = torch.cat((text_features, image_features), dim=1)

        # Predict Humor, Sarcasm, Offensive
        output = self.fc(combined_features)  # (batch_size, 3)
        return output


In [ ]:
model = MultimodalHumorModel().to(device)
criterion = nn.BCEWithLogitsLoss()  # Multi-label classification loss
optimizer = optim.Adam(model.parameters(), lr=2e-5)


In [ ]:
import numpy as np
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    all_labels = []
    all_preds = []

    for batch in dataloader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        images = batch["image"].to(device)
        labels = batch["labels"].to(device)  # Multi-label tensor

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask, images)

        # Compute loss
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        # Convert logits to binary predictions
        preds = (torch.sigmoid(outputs) > 0.5).int().cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(labels.cpu().numpy())

    # Convert lists to NumPy arrays
    all_labels = np.array(all_labels)
    all_preds = np.array(all_preds)

    # Compute metrics
    humor_f1 = f1_score(all_labels[:, 0], all_preds[:, 0], average="macro")
    sarcasm_f1 = f1_score(all_labels[:, 1], all_preds[:, 1], average="macro")
    offensive_f1 = f1_score(all_labels[:, 2], all_preds[:, 2], average="macro")
    avg_f1 = (humor_f1 + sarcasm_f1 + offensive_f1) / 3

    humor_acc = accuracy_score(all_labels[:, 0], all_preds[:, 0])
    sarcasm_acc = accuracy_score(all_labels[:, 1], all_preds[:, 1])
    offensive_acc = accuracy_score(all_labels[:, 2], all_preds[:, 2])

    humor_prec = precision_score(all_labels[:, 0], all_preds[:, 0], average="macro", zero_division=0)
    sarcasm_prec = precision_score(all_labels[:, 1], all_preds[:, 1], average="macro", zero_division=0)
    offensive_prec = precision_score(all_labels[:, 2], all_preds[:, 2], average="macro", zero_division=0)

    humor_rec = recall_score(all_labels[:, 0], all_preds[:, 0], average="macro", zero_division=0)
    sarcasm_rec = recall_score(all_labels[:, 1], all_preds[:, 1], average="macro", zero_division=0)
    offensive_rec = recall_score(all_labels[:, 2], all_preds[:, 2], average="macro", zero_division=0)

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(dataloader):.4f}")
    print(f"  - Humor    -> F1: {humor_f1:.4f}, Acc: {humor_acc:.4f}, Prec: {humor_prec:.4f}, Rec: {humor_rec:.4f}")
    print(f"  - Sarcasm  -> F1: {sarcasm_f1:.4f}, Acc: {sarcasm_acc:.4f}, Prec: {sarcasm_prec:.4f}, Rec: {sarcasm_rec:.4f}")
    print(f"  - Offensive-> F1: {offensive_f1:.4f}, Acc: {offensive_acc:.4f}, Prec: {offensive_prec:.4f}, Rec: {offensive_rec:.4f}")
    print(f"  - Avg F1 Score: {avg_f1:.4f}\n")


In [ ]:
torch.save(model.state_dict(), "multimodal_humor_model.pth")
print("Training completed and model saved successfully!")
